In [1]:
import FinanceDataReader as fdr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def getCloseData(ticker, start, end=None):
    """
    종가 데이터
    ticker: 종목 번호
    start: 시작일
    end: 마지막 날짜
    return: 종목의 종가 데이터
    """
    return fdr.DataReader(ticker, start, end)['Close']

def getDayReturn(closeDataSet):
    """
    개별종목 일별 수익률
    closeDataSet: 종가 데이터
    return: 종가 데이터의 일별 수익률
    """
    return (closeDataSet / closeDataSet.shift(1)).fillna(1)

def getCumulativeReturn(closeDataSet):
    """
    개별종목 누적수익률 == 자산흐름
    closeDataSet: 종가 데이터
    return:종가데이터 누적수익률
    """
    return closeDataSet / closeDataSet.iloc[0]

def getPortfolioResult(closeDataSet, weight=None):
    """
    포트폴리오 결과
    closeDataSet: 종가 데이터
    weight: 포트폴리오 개별자산 비중
    return: 포트폴리오 일간수익률, 누적수익률
    """
    # 개별종목 일별 수익률
    dayReturn = getDayReturn(closeDataSet)
    # 개별종목 누적 수익률
    cumulativeReturn = getCumulativeReturn(closeDataSet)
    # 자산별 비중. 기본값: 동일비중
    if not weight:
        weight = [1/len(closeDataSet.columns)] * len(closeDataSet.columns)
        
    # 포트폴리오 누적 수익률
    portfolioCumulativeReturn = (weight * cumulativeReturn).sum(axis=1)
    # 포트폴리오 일별 수익률
    portfolioDayReturn = (portfolioCumulativeReturn / portfolioCumulativeReturn.shift(1)).fillna(1)    
    return portfolioDayReturn, portfolioCumulativeReturn

def getEvaluation(cumulativeReturn):
    """
    cagr, dd, mdd
    투자 성과 지표
    """
    # cagr
    cagr = cumulativeReturn.iloc[-1] ** (252/len(cumulativeReturn))
    # mdd
    dd = (cumulativeReturn.cummax() - cumulativeReturn) / cumulativeReturn.cummax() * 100
    mdd= dd.max()
    
    print(f"최종 수익률: {cumulativeReturn.iloc[-1]}\ncagr: {cagr}\nmdd: {mdd}")

    return cagr, dd, mdd

def getRebalancingDate(closeDataSet, period="month"):
    """
    리밸런싱 일자 추출
    월별, 분기별, 연별
    """
    data = closeDataSet.copy()
    data = pd.DataFrame(data)
    data.index = pd.to_datetime(data.index)
    data['year'] = data.index.year
    data['month'] = data.index.month
    
    if period == "month":
        rebalancingDate = data.drop_duplicates(['year', 'month'], keep="last").index
        
    if period == "quarter":
        # 3 6 9 12월 말에 리밸런싱
        # np where 같은걸로 3, 6, 9, 12월 데이터만 가져오고
        # drop_duplicates keep last 하면 됌
        quarter = [3,6,9,12]
        data = data.loc[data['month'].isin(quarter)]
        rebalancingDate = data.drop_duplicates(['year', 'month'], keep="last").index
    
    if period == "year":
        rebalancingDate = data.drop_duplicates(['year'], keep="last").index
        
    return rebalancingDate

def getRebalancingPortfolioResult(closeDataSet, period = "month", weightDf=None):
    """
    리밸런싱 포트폴리오 결과
    closeDataSet: 종가 데이터
    weight: 포트폴리오 개별자산 비중
    return: 포트폴리오 일간수익률, 누적수익률
    """
    
    # 자산별 비중. 기본값: 동일비중
    if weightDf is None:
        rebalancingDate = getRebalancingDate(closeDataSet, period) # 리밸런싱 날짜
        weightDf = pd.DataFrame([[1/len(closeDataSet.columns)] * len(closeDataSet.columns)] * len(rebalancingDate),
                              index=rebalancingDate,
                              columns=closeDataSet.columns)
        
    rebalancingDate = weightDf.index # 리밸런싱 날짜
      
    portfolio = pd.DataFrame() # 빈 데이터 프레임 생성

    totalAsset = 1 # 총 자산, 초기값 1
    start = rebalancingDate[0] # 리밸런싱 날짜, 초기값 첫 투자일

    for end in rebalancingDate[1:]:
        weight = weightDf.loc[start] # 당월 리밸런싱 비율
        priceData = closeDataSet.loc[start:end] # 당월 가격 데이터
        cumReturn = getCumulativeReturn(priceData) # 당월 누적 수익률
        weightedCumReturn = weight * cumReturn # 당월 리밸런싱 비율이 반영된 누적 수익률
        netCumReturn = totalAsset * weightedCumReturn # 전월 투자 결과 반영

        start = end # start 갱신
        totalAsset = netCumReturn.iloc[-1].sum() # 총 자산 갱신
        portfolio = pd.concat([portfolio, netCumReturn]) # 매월 데이터 추가
    
    portfolio = portfolio.loc[~portfolio.index.duplicated(keep='last')] # 중복 데이터 제거
    portfolioCumulativeReturn = portfolio.sum(axis=1) # 포트폴리오 누적 수익률
    portfolioDayReturn = (portfolioCumulativeReturn / portfolioCumulativeReturn.shift(1)).fillna(1) # 포트폴리오 일간 수익률
    
    return portfolioDayReturn, portfolioCumulativeReturn

def getWeightByAvgMomentumScore(closeDataSet, n = 12):
    """
    평균 모멘텀 스코어를 기반으로 한 투자 비중 구하기
    closeDataSet: 종가 데이터
    n: 모멘텀 기간 1~n
    return: 투자비중 weight df, 평균모멘텀 스코어 df
    """
    avgMomentumScore = 0 # 평모스 초기값
    priceOnRebalDate = closeDataSet.loc[getRebalancingDate(closeDataSet)] # 리밸런싱 일자의 가격 데이터
    
    # 1 ~ n개월 모멘텀 스코어 합
    for i in range(1, n+1):
        avgMomentumScore = np.where(priceOnRebalDate / priceOnRebalDate.shift(i) > 1, 1, 0) + avgMomentumScore
        
    # 평모스 계산
    avgMomentumScore = pd.DataFrame(avgMomentumScore, index=priceOnRebalDate.index, columns=priceOnRebalDate.columns) # dataframe 형변환
    avgMomentumScore = avgMomentumScore / n
    
    # 모멘텀 스코어에 따른 weight 계산
    weight = avgMomentumScore.divide(avgMomentumScore.sum(axis=1), axis=0).fillna(0)
    # 투자 비중이 모두 0인 구간에서는 현금 보유
    weight['cash'] = np.where(weight.sum(axis=1) == 0, 1, 0)
    
    # 투자비중, 평모스 리턴
    return weight, avgMomentumScore

## DAA (VAA 업글 버전)

투자 전략

https://www.youtube.com/watch?v=2cNkblOxnFQ&t=679s

---

1. 공격자산, 수비자산 설정

공격자산 : SPY(S&P500), IWM(러셀 2000), QQQ(나스닥), VGK(유럽 주식), EWJ(일본 주식), VWO(개발도상국 주식), VNQ(미국 리츠), GSG(원자재), GLD(금), TLT(미국 장기채), HYG(하이일드 채권), LQD(회사채)
 
수비자산 : SHY(미국 단기국채), IEF(미국 중기국채), UST(미국 중기채 레버리지) -> TLT(미국 장기국채)

2. 카나리아 자산 설정

카나리아 자산 : VWO(개발도상국 주식), BND(미국 총채권)

3. 모멘텀 스코어 계산

(최근 1개월 수익률x12) + (최근 3개월 수익률x4) + (최근 6개월 수익률x2) + (최근 12개월 수익률x1)

4. 카나리아 자산 모멘텀 스코어가 모두 0 초과일 때 공격 자산 중 모멘텀 스코어가 가장 큰 2개 자산을 보유

5. 카나리아 자산 중 하나의 자산만 모멘텀 스코어0 초과일 때 모멘텀 스코어가 가장 큰 공격 자산 1개와 방어자산 1개

5. 카나리아 자산 모두 모멘텀스코어가 0 이하라면 수비 자산 중 모멘텀 스코어가 가장 큰 자산에 몰빵

In [3]:
# 원래는 UST(미국 중기채 레버리지)인데, UST 데이터가 부족하여 TLT(미국 장기채)로 대체
daaCol = ['SPY', "IWM", "QQQ", "VGK", "EWJ", "VWO", "VNQ", "GSG", "GLD", "TLT", "HYG", "LQD", "SHY", "IEF","BND"]
daaAttack = ['SPY', "IWM", "QQQ", "VGK", "EWJ", "VWO", "VNQ", "GSG", "GLD", "HYG", "LQD"]
daaDefense = [ "SHY", "IEF", "TLT"] 
daaCanary = ["VWO", "BND"]
daaData = closeDataSet[daaCol]
daaData

,SPY,IWM,QQQ,VGK,EWJ,VWO,VNQ,GSG,GLD,TLT,HYG,LQD,SHY,IEF,BND
2009-01-02,72.54897,42.24755,27.52616,25.02595,31.71652,18.37935,21.13848,29.61,86.23,82.52249,34.46859,63.88389,74.71815,75.41508,54.50851
2009-01-05,72.46313,42.39898,27.51729,24.76346,30.92608,18.67543,20.74913,30.63,84.48,80.39471,36.13357,64.32201,74.86900,75.28307,54.27989
2009-01-06,72.94700,43.11404,27.79228,25.21161,30.62966,19.00852,21.78539,31.20,85.13,79.58615,36.43835,64.42996,74.90449,75.23648,53.93349
2009-01-07,70.76178,41.84376,27.00278,24.75066,30.33324,18.00184,21.05462,29.02,82.75,79.89822,35.77874,63.89024,74.85125,75.25201,54.09976
2009-01-08,71.05054,42.04565,27.28665,25.02595,30.76140,17.89081,20.89888,28.62,84.46,79.83439,35.26014,64.34741,74.86900,75.45390,54.00970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-27,388.59000,175.74000,292.45000,53.97000,53.23000,42.22000,92.98000,23.94,169.90,111.60000,74.58000,109.08000,82.55000,100.58000,74.47000
2022-06-28,380.65000,172.44000,283.54000,53.44000,53.15000,41.94000,91.85000,24.23,169.62,112.12000,73.76000,109.05000,82.53000,100.71000,74.51000
2022-06-29,380.34000,170.69000,283.80000,53.13000,52.91000,41.83000,91.28000,23.79,169.49,113.87000,73.65000,109.70000,82.63000,101.52000,74.94000
2022-06-30,377.25000,169.36000,280.28000,52.82000,52.83000,41.65000,91.11000,23.10,168.46,114.87000,73.61000,110.03000,82.79000,102.30000,75.26000


### 모멘텀 스코어 계산

(최근 1개월 수익률x12) + (최근 3개월 수익률x4) + (최근 6개월 수익률x2) + (최근 12개월 수익률x1)

In [4]:
rebalDate = getRebalancingDate(daaData)
rebalDate

DatetimeIndex(['2009-01-30', '2009-02-27', '2009-03-31', '2009-04-30',
               '2009-05-29', '2009-06-30', '2009-07-31', '2009-08-31',
               '2009-09-30', '2009-10-30',
               ...
               '2021-10-29', '2021-11-30', '2021-12-31', '2022-01-31',
               '2022-02-28', '2022-03-31', '2022-04-29', '2022-05-31',
               '2022-06-30', '2022-07-01'],
              dtype='datetime64[ns]', length=163, freq=None)

In [5]:
rebalDate = getRebalancingDate(daaData)
daaDataOnRebalDate = daaData.loc[rebalDate]

momentum1 = daaDataOnRebalDate / daaDataOnRebalDate.shift(1) -1
momentum3 = daaDataOnRebalDate / daaDataOnRebalDate.shift(3) - 1
momentum6 = daaDataOnRebalDate / daaDataOnRebalDate.shift(6) -1
momentum12 = daaDataOnRebalDate / daaDataOnRebalDate.shift(12) -1

momentumScore = 12*momentum1 + 4*momentum3 + 2*momentum6 + momentum12
momentumScore.dropna(inplace=True)
momentumScore

,SPY,IWM,QQQ,VGK,EWJ,VWO,VNQ,GSG,GLD,TLT,HYG,LQD,SHY,IEF,BND
2010-01-29,0.256459,0.384538,0.065752,-0.308116,0.477950,0.317359,0.670668,-1.141503,0.413458,0.126380,0.335681,0.385464,0.151217,0.265636,0.339963
2010-02-26,1.141328,1.714132,1.547409,0.224417,0.649499,1.283355,2.260594,0.965066,0.702795,-0.336461,1.029482,0.289896,0.060500,-0.022881,0.181578
2010-03-31,1.685561,2.240035,2.023380,1.336952,1.351332,2.125673,3.128230,0.327873,0.419646,-0.506962,0.925043,0.397659,0.014054,-0.134470,0.126598
2010-04-30,1.331963,2.516934,1.731827,-0.055818,0.600274,1.182248,3.266433,1.136831,1.633006,0.373176,0.958518,0.614363,0.070622,0.239552,0.295530
2010-05-28,-0.772358,-0.041288,-0.412609,-2.082185,-1.152256,-1.070846,0.817873,-2.360321,1.003379,0.973092,-0.337878,0.045539,0.104322,0.578877,0.335825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-31,0.464272,-0.386608,0.352562,-0.459067,-0.913363,-0.957656,0.736465,3.664342,0.708870,-1.291040,-0.574032,-0.963597,-0.371985,-0.939348,-0.786000
2022-04-29,-1.605514,-2.066041,-2.629701,-1.610959,-1.936030,-1.662277,-0.614165,2.721103,0.152494,-2.308065,-1.122855,-1.749928,-0.268668,-1.169049,-1.149679
2022-05-31,-0.390895,-0.809185,-1.144835,-0.170248,-0.548811,-0.705855,-0.842374,3.259768,-0.526166,-1.573824,-0.273550,-0.542052,-0.082974,-0.565008,-0.479735
2022-06-30,-2.225870,-2.471576,-2.797751,-2.674871,-2.203627,-1.484589,-2.135439,0.285607,-0.469991,-1.347097,-1.777399,-1.340754,-0.210631,-0.644808,-0.782449


**DAA 투자비중 계산**

In [6]:
isAttack = (momentumScore[daaCanary] > 0).sum(axis=1)
isAttack

2010-01-29    2
2010-02-26    2
2010-03-31    2
2010-04-30    2
2010-05-28    1
             ..
2022-03-31    0
2022-04-29    0
2022-05-31    0
2022-06-30    0
2022-07-01    0
Length: 151, dtype: int64

In [12]:
s = pd.Series([10, 12, 15], index=['a', 'b', 'c'], name="dummy series")
s

a    10
b    12
c    15
Name: dummy series, dtype: int64

In [14]:
s.nlargest(n=2)

c    15
b    12
Name: dummy series, dtype: int64

In [18]:
s.isin([10, 12])

a     True
b     True
c    False
Name: dummy series, dtype: bool

In [7]:
def applyGetDAAWeight(row):
    if isAttack[row.name] == 2:
        # 카나리아 자산 모멘텀 스코어가 모두 0 초과일 때 공격 자산 중 모멘텀 스코어가 가장 큰 2개 자산을 보유
        top2 = row[daaAttack].nlargest(n=2).index
        result = pd.Series(row.index.isin(top2), index=row.index, name=row.name).astype(int).replace(1, 0.5)
        return result
    
    if isAttack[row.name] == 1:
        # 카나리아 자산 중 하나의 자산만 모멘텀 스코어0 초과일 때 모멘텀 스코어가 가장 큰 공격 자산 1개와 방어자산 1개
        topAttack = row[daaAttack].idxmax()
        topDefense = row[daaDefense].idxmax()
        result = pd.Series(row.index.isin([topAttack, topDefense]), index=row.index, name=row.name).astype(int).replace(1, 0.5)
        return result 
        
    # 카나리아 자산 모두 모멘텀스코어가 0 이하라면 수비 자산 중 모멘텀 스코어가 가장 큰 자산에 몰빵
    return pd.Series(row.index == row[daaDefense].idxmax(), index=row.index, name=row.name).astype(int)
    

daaWeight = momentumScore.apply(applyGetDAAWeight, axis=1)
daaWeight = daaWeight.loc['2010-12-31':]
daaWeight

,SPY,IWM,QQQ,VGK,EWJ,VWO,VNQ,GSG,GLD,TLT,HYG,LQD,SHY,IEF,BND
2010-12-31,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0
2011-01-31,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-02-28,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-03-31,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-04-29,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2022-04-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2022-05-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2022-06-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [8]:
daaDayReturn, daaCumReturn = getRebalancingPortfolioResult(closeDataSet=daaData, weightDf=daaWeight)
daaCagr, daaDD, daaMDD = getEvaluation(daaCumReturn)

최종 수익률: 2.6483399282354974
cagr: 1.0884749495548425
mdd: 18.252476001245633


## DAA 비중 계산 함수화

In [9]:
def getDAAWeight(closeDataSet):
    
    daaCol = ['SPY', "IWM", "QQQ", "VGK", "EWJ", "VWO", "VNQ", "GSG", "GLD", "TLT", "HYG", "LQD", "SHY", "IEF","BND"]
    daaAttack = ['SPY', "IWM", "QQQ", "VGK", "EWJ", "VWO", "VNQ", "GSG", "GLD", "HYG", "LQD"]

    # UST 데이터 시점 문제로 TLT로 대체
    daaDefense = [ "SHY", "IEF", "TLT"] 
    daaCanary = ["VWO", "BND"]

    daaData = closeDataSet[daaCol].copy()
    daaData.dropna(inplace=True)
    
    # 모멘텀 스코어 계산
    rebalDate = getRebalancingDate(daaData)
    daaDataOnRebalDate = daaData.loc[rebalDate]

    momentum1 = daaDataOnRebalDate / daaDataOnRebalDate.shift(1) -1
    momentum3 = daaDataOnRebalDate / daaDataOnRebalDate.shift(3) - 1
    momentum6 = daaDataOnRebalDate / daaDataOnRebalDate.shift(6) -1
    momentum12 = daaDataOnRebalDate / daaDataOnRebalDate.shift(12) -1

    momentumScore = 12*momentum1 + 4*momentum3 + 2*momentum6 + momentum12
    momentumScore.dropna(inplace=True)
    
    # 카나리아 자산 모멘텀 스코어가 모두 0 초과일 때 공격 자산 중 모멘텀 스코어가 가장 큰 2개 자산을 보유
    # 카나리아 자산 중 하나의 자산만 모멘텀 스코어0 초과일 때 모멘텀 스코어가 가장 큰 공격 자산 1개와 방어자산 1개
    # 카나리아 자산 모두 모멘텀스코어가 0 이하라면 수비 자산 중 모멘텀 스코어가 가장 큰 자산에 몰빵
    isAttack = (momentumScore[daaCanary] > 0).sum(axis=1)
    daaWeight = momentumScore.apply(applyGetDAAWeight, axis=1, args=(isAttack,))
    return daaWeight
    
def applyGetDAAWeight(row, isAttack):
    daaCol = ['SPY', "IWM", "QQQ", "VGK", "EWJ", "VWO", "VNQ", "GSG", "GLD", "TLT", "HYG", "LQD", "SHY", "IEF","BND"]
    daaAttack = ['SPY', "IWM", "QQQ", "VGK", "EWJ", "VWO", "VNQ", "GSG", "GLD", "HYG", "LQD"]

    # UST 데이터 시점 문제로 TLT로 대체
    daaDefense = [ "SHY", "IEF", "TLT"] 
    daaCanary = ["VWO", "BND"]
    
    if isAttack[row.name] == 2:
        # 카나리아 자산 모멘텀 스코어가 모두 0 초과일 때 공격 자산 중 모멘텀 스코어가 가장 큰 2개 자산을 보유
        top2 = row[daaAttack].nlargest(n=2).index
        result = pd.Series(row.index.isin(top2), index=row.index, name=row.name).astype(int).replace(1, 0.5)
        return result
    
    if isAttack[row.name] == 1:
        # 카나리아 자산 중 하나의 자산만 모멘텀 스코어0 초과일 때 모멘텀 스코어가 가장 큰 공격 자산 1개와 방어자산 1개
        topAttack = row[daaAttack].idxmax()
        topDefense = row[daaDefense].idxmax()
        result = pd.Series(row.index.isin([topAttack, topDefense]), index=row.index, name=row.name).astype(int).replace(1, 0.5)
        return result 
        
    # 카나리아 자산 모두 모멘텀스코어가 0 이하라면 수비 자산 중 모멘텀 스코어가 가장 큰 자산에 몰빵
    return pd.Series(row.index == row[daaDefense].idxmax(), index=row.index, name=row.name).astype(int)

In [10]:
daaWeight = getDAAWeight(closeDataSet)
daaWeight

,SPY,IWM,QQQ,VGK,EWJ,VWO,VNQ,GSG,GLD,TLT,HYG,LQD,SHY,IEF,BND
2010-01-29,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-02-26,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-03-31,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-04-30,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-05-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2022-04-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2022-05-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2022-06-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
